## P8: 
### "Around the world in 80 days"

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt 
import folium 
import math
from sklearn.metrics.pairwise import haversine_distances
from sklearn.neighbors import BallTree
from typing import List
from collections import defaultdict 

import networkx as nx

In [ ]:
#Read the dataset 

fields = ["city","lat","lng","country","population"]
df = pd.read_excel('/Users/ariannabonani/Desktop/Arianna/DocumentiMaster/Python/worldcities.xlsx',usecols=fields)

number_city = len(df)



In [ ]:
#Calculate two useful values of longitude: the largest value and London longitude 

max_lng = max(df.lng.values)

max_lat=max(df.lat.values)

#calcolo la longitudine di Londra
lng_london = [x for city, country, x in df[['city','country','lng']].values if city == "London" and country == "United Kingdom"][0]

                

In [ ]:
#Do a longitude normalization as 
# max_lng+||xlng|-max_lng|+ |lng_london|.

#It is necessary to move only to the East, so if a city A has a lower 
#longitude than London a faster movement to the left is forbidden. 
#Only by circling the world we are allowed to reach A.

#Add the found values in a new column to the dataframe named 'lng_norm' 

    
lng_norm = []

#scalo la latitudine per avere lo stesso ordine di grandezza della longitudine



for i in range( 0, number_city):
    if df.lng.values[i] < lng_london:
        lng_norm.append(max_lng + abs(abs(df.lng.values[i])-max_lng) + abs(lng_london))
    else:
        lng_norm.append(df.lng.values[i]+ abs(lng_london))
   

df['nlng'] = lng_norm  

#riscalo anche la longitudine

max_nlng =  max(df.nlng.values)


In [ ]:

#Sort the cities in the dataframe by increasing longitude

#Add a new record at the end of the dataframe:
#London values are copied because it is both the departure and arrival of the tour.
#London as the end of the travel has a longitude of 360° compared 
#to London as the start.

#Calculate the number of city in the dataset 'number_city'

df.sort_values(by=['nlng'], inplace = True)
df.loc[number_city] = df.iloc[0]
df.nlng[number_city] = 360


#riscalo anche la longitudine 

number_city += 1

In [ ]:
# Get ready an empty dictionary to save:
# -- each city according to the respective row position in the dataframe
#(London is t<he first row in the dataframe, and it is indexed with 0);
# -- a list containing tuples, each filled with one of the three closest cities 
# and the time to reach it

C = defaultdict(lambda: [])

# Get ready an empty directed graph.
# It will be filled with tuples of edges and their weight


G = nx.DiGraph()

In [ ]:
#-----------------I-----------------
df['nlat'] = df.lat.values / max(df.lat.values)
df.nlng = df.nlng.values/ 180

# Fill up the dictionary and build the graph: first method -- standard version


for i in range(0,number_city-1):
        
    #extract the three indices corresponding to the three closest cities calculated the Euclidean Distance
        
    distances = haversine_distances(df[['nlat','nlng']].values[i].reshape(1,-1), df[['nlat','nlng']].values[i+1:])  
    
    
    #sorts the array of distances in ascending order starting from the first city 
    #reachable (df.city.iloc [i+1:]) from the city of departure df.city.iloc [i]
    index= np.argsort(distances[0][:])[0:3]

    
    #For the first nearest city it takes 2 hours, for the second 4 hours and for the third 8 hours.
    #If the arrival city has more than 200.000 inhabitants, add two hours.
    #If the arrival and starting cities are in different Countries, add two hours.
    
    
    for it,j in enumerate(index):
        time= pow(2,(it+1))
        if df.population.values[j+1+i] > 200000:
            time +=2
        elif df.country.values[j+1+i] != df.country.values[j+1+i]:
            time += 2  
        C[i].append((j+1+i,time))
        G.add_weighted_edges_from([(i, j+1+i , time)])      
 
    #add a blank value for arrival-London, the finishing line

C[number_city-1] = [] 



In [ ]:
#-----------------II-----------------


# Fill up the dictionary and build the graph: third method with KNeighbors clustering 

df['nlat'] = df.lat.values / max(df.lat.values)
df.nlng = df.nlng.values/ 180

# Setup distance queries (points for which we want to find nearest neighbors)
tree = BallTree(df[['nlat', 'nlng']], metric='haversine')
# Setup Balltree using df as reference dataset
distances, indices = tree.query(df[['nlat', 'nlng']], k = int(number_city-1))


for i in range(0,number_city-1):
        
    # Find closest city in reference dataset use k = 3 for 3 closest neighbors
    index=[]
    j=0
    while len(index)<3 and j<int(number_city-1):
        if indices[i][j] > i:
            index.append(indices[i][j])
        j += 1    
        
        
    for it,j in enumerate(index):
        time= pow(2,(it+1))        
        if df.population.values[j] > 200000:
            time +=2
        elif df.country.values[j] != df.country.values[j]:
            time += 2
        try:    
            C[i].append((j,time))
        except KeyError:
            C[i]= [(j,time)]
    
        G.add_weighted_edges_from([(i, j , time)])      
       
C[number_city-1] = [] 

In [ ]:
#-----------------III-----------------


#Build an empty grid using the 'grid' dictionary. 
# 
#Each cell can be accessed via two keys, relating to a measure of latitude and longitude. 
#Each city in the dataset belongs to only one cell.



def grid():
     

    df["grid_lat"] = [int(x//1) for x in df.nlat.values]
    df["grid_nlng"] = [int(x//2) for x in df.nlng.values]

    # 'grid' dictionary contains the lists of cities that belong to the same cell

    grid= defaultdict(lambda: defaultdict(lambda: []))


    for i, grid_cord in enumerate(df[['grid_lat','grid_nlng']].values):
        grid[grid_cord[0]][grid_cord[1]].append(i)
        
    return grid


In [ ]:
# << give_index >> returns the elements of the first list 'grid_plus' that have an index contained in the array 'indices'

def give_index(grid_plus: List[int] ,indices):
    grid=[]
    for i in indices:
        grid.append(grid_plus[i])
        
    return grid    

In [ ]:
# << give_basic_frame >>  selects the grid corresponding to the city with 'grid_lat' and 'grid_nlng' of the city and returns
# the list of all the cities to calculate the distance from the i-th city

def give_basic_frame(grid_lat, grid_nlng, i):
    
         
    grid_plus=[]
    for glat in range(grid_lat-1, grid_lat+2):
        grid_plus += grid[glat][grid_nlng] 
    #seleziono solo le città con una longitudine maggiore della città i-esima
    grid_plus = [city for city in grid_plus if df.nlng.values[city] > df.nlng.values[i]]
    for glat in range(grid_lat-1, grid_lat+2):
        grid_plus += grid[glat][grid_nlng+1] 
    return grid_plus


#  <<give_frame>> selects more cities in order to calculate the distance from the i-th city if the length of 'grid_plus' is less than 3

def give_frame(grid_plus,grid_lat, grid_nlng, i):
    
    u=2
    grid_plus_plus=[]
    while len(grid_plus)< 3:
        grid_plus += grid[grid_lat-u][grid_nlng] + grid[grid_lat+u][grid_nlng]
        #Select the reachable cities, i.e. cities with a longitude greater than the longitude of the i-th city        
        grid_plus = [city for city in grid_plus if df.nlng.values[city] > df.nlng.values[i]]

        for glng in range(grid_nlng+1,grid_nlng+u):
            grid_plus += grid[grid_lat-u][glng]
            grid_plus += grid[grid_lat+u][glng]

        for glat in range(grid_lat-u,grid_lat+u+1):
            grid_plus += grid[glat][grid_nlng+u]
        u += 1 
      
    grid_plus_plus= grid[grid_lat-u][grid_nlng] + grid[grid_lat+u][grid_nlng]
        #Select the reachable cities, i.e. cities with a longitude greater than the longitude of the i-th city        
    grid_plus_plus = [city for city in grid_plus_plus if df.nlng.values[city] > df.nlng.values[i]]

    for glng in range(grid_nlng+1,grid_nlng+u):
        grid_plus_plus += grid[grid_lat-u][glng]
        grid_plus_plus += grid[grid_lat+u][glng]

    for glat in range(grid_lat-u,grid_lat+u+1):
        grid_plus_plus += grid[glat][grid_nlng+u]
        
    return grid_plus, grid_plus_plus

In [ ]:
# Fill up the dictionary and build the graph with a grid 180x180: third method

df['nlat']=90-df.lat.values

grid= grid()

df.nlat = df.lat.values/ max_lat
df.nlng = df.nlng.values/ 180

for i in range(0, number_city-1):
    
    if i==number_city-3:
        grid_plus=[number_city-2, number_city-1]
        distances = haversine_distances(df[['nlat','nlng']].values[i].reshape(1,-1), df[['nlat','nlng']].values[grid_plus])
    elif i== number_city-2:
        grid_plus=[number_city-1]
        distances = haversine_distances(df[['nlat','nlng']].values[i].reshape(1,-1), df[['nlat','nlng']].values[grid_plus].reshape(1,-1))
    else:    

        grid_lat, grid_nlng = df[['grid_lat','grid_nlng']].values[i]

        grid_plus = give_basic_frame(grid_lat, grid_nlng, i)
        
        
        grid_plus, grid_plus_plus =give_frame(grid_plus,grid_lat,grid_nlng, i)

        #Calculate the Euclidean distance only between the i-th city and the cities contained in the appropriate frame
        grid_plus += grid_plus_plus

        distances = haversine_distances(df[['nlat','nlng']].values[i].reshape(1,-1), df[['nlat','nlng']].values[grid_plus])

    indices= np.argsort(distances[0][:])[0:3]
    index= give_index(grid_plus, indices)
    
                    
    for it,j in enumerate(index):
        time= pow(2,(it+1))
        if df.population.values[j] > 200000:
            time +=2
        elif df.country.values[j] != df.country.values[j]:
            time += 2
        C[i].append((j,time))
        G.add_weighted_edges_from([(i, j , time)])      
        
        
#add a blank value for arrival-London, the finishing line    
C[number_city-1] = [] 

In [ ]:
#call the default Dijkstra algorithm for calculating the path of least weight

# returns the index of cities reached by the path

dijkstra_path=nx.dijkstra_path(G, source=0, target=number_city-1, weight='weight')

In [ ]:
#To find the fastest way, we analyze the 'pred' vector backwards
# and insert the indexes of the corresponding cities in 'path'.

#Remember: London (start) is indexed with 0 and London (arrival) with 26569

#The function needs the index of the starting and ending node 
#and the predecessors-vector calculated by my_Dijkstra algorithm

def my_path(s:int,t:int, pred):
    path=[]
    path.insert(0,t)
    
    i=int(pred[t])
    path.insert(0,i)
    while i!=s:
        i=int(pred[i])
        path.insert(0,i) 
    return path


In [ ]:
#Implementation of my_Dijkstra's algorithm 

def my_Dijkstra(C,s:int,t:int):
    
    #PART 1: initialize useful arrays

    # d = array containing the distances of each node from the starting node
    # d[3] cointains the distance from London to the city indexed with 3 in the dataframe 
    #(the fouth city in the dataframe).

    # pred = array of predecessors, through which we reconstruct the fastest path

    # L = list of nodes not yet reached by the algorithm.
    #They are not in S, but reachable only through a shift 
    #from the nodes in S. 

    # S = list of nodes already been screened by algorithm
    
    L=[]
    d=np.zeros(t+1)
    pred= -1*np.ones(t+1) 


    for v in range(0,t+1): 
        d[v]= float("inf")

        
    d[s]=0
    pred[s]=int(s)
    L.append((s,d[s]))
    #Start the algorithm loop

    while L!= []:

        L=sorted(L,key=lambda x: x[-1]) 
        #ordino rispetto al secondo: i vuole essere il nodo con peso minore d a partire da quello sorgente
        i= L[0][0]
        L.remove(L[0]) 
        for j,v in enumerate(C[i]):
            if d[v[0]] > d[i]+v[1]:
                if d[v[0]] == float("inf"):
                    L.append((v[0],d[i]+ v[1]))
                d[v[0]]= d[i]+ v[1]
                pred[v[0]] = int(i) 
                
    #Call the function  "my_path". It returns the path          
                
    path=my_path(s,t,pred) 
    return path


In [ ]:
#give the path from London to London 
my_path = my_Dijkstra(C,0,number_city-1) 


In [ ]:

#The function returns the time for travelling around the World and requires the number of nodes, their wheigh 
#and the fastes path around the world from the output of Dijkstra's algorith

def my_path_lenght(number_nodes:int,C,path):
  
    # Build time matrix: we imagine a graph where nodes are the cities 
    #and the time-weighted arcs are the possible connections.
    
    T=np.zeros((number_nodes,number_nodes))

    for i in range(0,number_nodes):
        for j,v in enumerate(C[i]):
            T[i][v[0]]= C[i][j][1]
    #Find the minimum time to travel the optimal path.

    travel_time=0

    for i in range(0,len(path)-1):
        travel_time += T[path[i]][path[i+1]]
        
    day_travel=travel_time/24
    
    hour_travel= day_travel - int(day_travel)
    
    
    minutes_travel = hour_travel - int(hour_travel)

    
    time_string= str(int(day_travel)) + " days : " + str(int(hour_travel * 24)) + " hours : "+ str(int((minutes_travel * 60)))+ " minutes"
    
    return time_string

In [ ]:
#print the length of the path obtained both with the function <<my_Dijkstra>> and with the algorithm of Dijkstra 
# implemented by Pyhton

dijkstra_path_length= nx.dijkstra_path_length(G, source=0, target=number_city-1, weight='weight')/24
 
dijkstra_hour_travel= dijkstra_path_length - int(dijkstra_path_length)
    
dijkstra_minutes_travel = dijkstra_hour_travel - int(dijkstra_hour_travel)

print("Il giro del Mondo impiega: " ,int(dijkstra_path_length), " days : ", int(dijkstra_hour_travel * 24), " hours : ", int(dijkstra_minutes_travel * 60), " minutes")

my_path_lenght = my_path_lenght(number_city,C,my_path)
print("Il giro del Mondo impiega: " ,my_path_lenght)



In [ ]:
# Build the map with all the cities in the direction of the path
# Build the map with the reached cities in the direction of the path

city = pd.DataFrame([df['city'].iloc[my_path],df['lat'].iloc[my_path],df['lng'].iloc[my_path]]).T

city_of_world= pd.DataFrame([df['city'],df['lat'],df['lng']]).T

geometry=gpd.points_from_xy(df['lng'].iloc[my_path], df['lat'].iloc[my_path])

geometry_of_world=gpd.points_from_xy(df['lng'], df['lat'])


world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


fig, ax= plt.subplots(figsize=(70, 35), nrows=2, ncols=1)

world.plot(ax=ax[0], color='white', edgecolor='black')
world.plot(ax=ax[1], color='white', edgecolor='black')

gdf_of_world = gpd.GeoDataFrame(city_of_world, geometry=geometry_of_world)
gdf = gpd.GeoDataFrame(city, geometry=geometry)


gdf_of_world.plot(ax=ax[0], color='pink', marker= '.', markersize=200)

gdf.plot(ax=ax[1], color='red', marker= 9, markersize=1000)

plt.show()

In [ ]:
# Build the map with the cities of the tour pointed at flags 

#import requests

resolution, width, height = 75, 12, 9

map = folium.Map(location=[51,0],zoom_start=1)

folium.Marker([df['lat'].iloc[0],df['lng'].iloc[0]], popup=df['city'].iloc[0],icon=folium.Icon(color='pink',icon="flag")).add_to(map)
for i in my_path[1:-2]:
    folium.Marker([df['lat'].iloc[i],df['lng'].iloc[i]], popup=df['city'].iloc[i],icon=folium.Icon(color='blue',icon='flag')).add_to(map)
map